In [ ]:
import pandas as pd
from PIL import Image
from google.cloud import storage

In [ ]:
# Source : https://www.kaggle.com/datasets/nagasai524/mini-coco2014-dataset-for-image-captioning

# captions_data_path = '$LOCAL_JSON_PATH'
# images_base_path = '$LOCAL_IMAGE_PATH'

# df = pd.read_json(captions_data_path)

# sampling small part of the dataset
# df_small = df.iloc[:100]

# def path_converter(base_path, num):
#     num_str = str(num)
#     num_zeros = 12 - len(num_str)

#     return base_path + '0' * num_zeros + num_str + '.jpg'

# df_small['image_path'] = df_small['image_id'].apply(lambda x: path_converter(images_base_path, x))

In [ ]:
# Manually sampled images & generated questions
data = pd.read_csv('/Users/lexha/Documents/work/codes/vapo-examples/2-4/sample_data/VAPO-use-case-2-4-image-qa-data-sampled.csv')

In [ ]:
# idx=0
# print(data['question'][idx])
# print(data['answer'][idx])

In [ ]:
# Train-test split
test_size=14

train_data = data[:-test_size]
test_data = data[-test_size:]

print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.to_csv('./sample_data/train-2-4.csv', index=False)
test_data.to_csv('./sample_data/test-2-4.csv', index=False)

### Upload to GCS

In [ ]:
# !gcloud auth application-default login

In [ ]:
# Upload to GCS bucket
BUCKET_NAME = '$YOUR_BUCKET_NAME'
image_folder_path = 'inputs/image'
gcs_base_path = f'gs://{BUCKET_NAME}/{image_folder_path}/'

In [ ]:
train_data['image'] = train_data['image_path'].apply(lambda x: gcs_base_path + x.split('/')[-1])
train_data.rename(columns={'answer': 'target'}, inplace=True)

In [ ]:
def upload_df_to_gcs_as_jsonl(df: pd.DataFrame, bucket_name: str, destination_blob_name: str):
    jsonl_string = df.to_json(orient='records', lines=True, force_ascii=False)
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_string(jsonl_string, content_type='application/json')

    print(f"Uploaded : {bucket_name} / {destination_blob_name}")

In [ ]:
train_data_path = 'inputs/train_image.jsonl'
upload_df_to_gcs_as_jsonl(train_data[['question', 'target', 'image']], BUCKET_NAME, train_data_path)

In [ ]:
# Upload image files to GCS
def upload_img_to_gcs(bucket_name, source_file_name, destination_blob_name, verbose=False):
    try:
        storage_client = storage.Client()
    except Exception as e:
        print(f"Error: {e}")
        return

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    if verbose:
        print(f"File upload started: '{source_file_name}' -> 'gs://{bucket_name}/{destination_blob_name}'")

    try:
        blob.upload_from_filename(source_file_name, content_type='image/jpeg')

        if verbose:    
            print(f"File '{source_file_name}' Uploaded to '{destination_blob_name}'")
        
    except Exception as e:
        print(f"Error when Uploading: {e}")

In [ ]:
for idx in range(len(train_data)):
    gcs_file_path = train_data['image'][idx]
    local_file_path = train_data['image_path'][idx]
    upload_img_to_gcs(BUCKET_NAME, local_file_path, gcs_file_path, verbose=True)